In [1]:

%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))




Found GPU at: /device:GPU:0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

import numpy as np 
import pandas as pd

import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator 
from PIL import Image 
from glob import glob
import pprint



import os
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras import layers
from tensorflow.keras import Model

from tensorflow.keras.applications.resnet import ResNet50, ResNet101
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam, Nadam
import pathlib
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions

for dirname, _, filenames in os.walk('./datas/fruits-360/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [4]:
#train_path = pathlib.Path('./datas/fruits-360/Training/')
#test_path = pathlib.Path('./datas/fruits-360/Test/')

train_path = pathlib.Path('./datas/fruits-360-mini/train/')
test_path = pathlib.Path('./datas/fruits-360-mini/test/')

#train_path = ('./drive/MyDrive/fruits-360-mini/train/')
#test_path = ('./drive/MyDrive/fruits-360-mini/test/')

In [5]:
subs_train_label = [x.split('/')[-1] for x in os.listdir(train_path)]
subs_test_label = [x.split('/')[-1] for x in os.listdir(test_path)]
#subs_train = [x[0] for x in os.walk(train_path)]
#subs_test = [x[0] for x in os.walk(test_path)]


In [6]:
subs_test_label

['mango', 'orange', 'apple', 'banana', 'strawberry']

In [7]:
subs_train_labels = list(filter(None, subs_train_label))
subs_test_labels = list(filter(None, subs_test_label))
#subs_train_paths = list(filter(None, subs_train))[1:]
#subs_test_paths = list(filter(None, subs_test))[1:]


print(len(subs_train_labels))
print(len(subs_test_labels))
#print(len(subs_train_paths))
#print(len(subs_test_paths))
print(set(subs_train_label) == set(subs_test_label))

5
5
True


In [8]:
labels = sorted(subs_train_labels)

In [9]:
labels[0]

'apple'

In [10]:
#x = load_img(str(train_path) + '/' + labels[0] + "/10_100.jpg")
x = load_img(str(train_path) + '/' + labels[0] + "/image_0.jpg")
img_shape = img_to_array(x).shape
img_shape = [224,224,3]
print(img_shape)

[224, 224, 3]


In [11]:
#fig = plt.figure(figsize =(15,8))
#for i in range(25):
#    ax = fig.add_subplot(5,5,i+1,xticks=[],yticks=[])
#    plt.axis("off")
#    plt.title(labels[i])
#    ax.imshow(load_img(str(train_path) + '/' + labels[i] +"/10_100.jpg"))

In [12]:
pre_trained_model = ResNet101(input_shape = img_shape, 
                                include_top = False, # without FC
                                weights = 'imagenet')

In [13]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [14]:
class myCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.95):
            print("\nReached 95% accuracy so cancelling training!")
            self.model.stop_training = True

In [15]:
#tf.keras.backend.clear_session()



# Prepare for FC connexion
#model = Sequential([pre_trained_model, 
#            layers.GlobalAveragePooling2D(),
#            layers.Dropout(rate=0.5),
#            layers.Dense(1024),
#            layers.Dropout(rate=0.5),
#            layers.Dense(5),
#            layers.Softmax()])

x = pre_trained_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x) 
x = layers.Dense(len(labels), activation='softmax')(x)           

model = Model(pre_trained_model.input, x) 

model.compile(optimizer = Adam(lr=0.001), 
              loss = tf.keras.losses.CategoricalCrossentropy(), 
              metrics = ['accuracy'])




In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [17]:
img_shape[:2]

[224, 224]

In [18]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator( rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(train_path,
                                                    batch_size = 32,
                                                    class_mode  = 'categorical',
                                                    target_size = img_shape[:2])   

validation_generator =  test_datagen.flow_from_directory( test_path,
                                                          batch_size  = 32,
                                                          class_mode  = 'categorical',
                                                          target_size = img_shape[:2])

Found 1500 images belonging to 5 classes.
Found 500 images belonging to 5 classes.


In [ ]:
callbacks = myCallback()
history = model.fit_generator(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 1500//32,
            epochs = 50,
            validation_steps = 500//32,
            verbose = 2,
            callbacks=[callbacks])


Epoch 1/50
46/46 - 33s - loss: 1.7862 - accuracy: 0.2350 - val_loss: 1.6473 - val_accuracy: 0.2479
Epoch 2/50


In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()